<a href="https://colab.research.google.com/github/haticekck/object_detection/blob/main/faster-rcnn_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!unzip /content/drive/MyDrive/data2.zip -d data2

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: data2/data/rcnn_labels/train/MVI_40871_img01234.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40871_img01234.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40213_img00015.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40213_img00015.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40131_img00113.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40131_img00113.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40991_img00571.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40991_img00571.txt  
  inflating: data2/data/rcnn_labels/train/MVI_20064_img00351.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_20064_img00351.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40162_img01430.txt  
  inflating: data2/__MACOSX/data/rcnn_labels/train/._MVI_40162_img01430.txt  
  inflating: data2/data/rcnn_labels/train/MVI_40991_img00217.txt  
  inflating: dat

In [9]:
import os
import torch
from PIL import Image
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
from tqdm import tqdm

# Cihaz
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Modeli başlat (COCO değil)
model = fasterrcnn_resnet50_fpn(pretrained=False)

# Sınıf sayını belirle
num_classes = 5  # 4 sınıf + 1 background

# Kendi sınıf sayına göre box predictor’ı güncelle
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Eğitilmiş ağırlıkları yükle
model.load_state_dict(torch.load('/content/drive/MyDrive/fasterrcnn_final.pth', map_location=device))
model.to(device)
model.eval()

# === Klasörler ===
image_folder = '/content/data2/data/images/val'
label_folder = '/content/data2/data/rcnn_labels/val'  # .txt dosyalarının olduğu klasör

# === IoU hesaplayıcı ===
def compute_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter + 1e-6
    return inter / union

# === Her görselin label dosyasını oku ===
def load_labels(label_path):
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = list(map(float, line.strip().split()))
            x1, y1, x2, y2 = parts[:4]
            bboxes.append([x1, y1, x2, y2])
    return bboxes

# === Transform ===
transform = T.Compose([T.ToTensor()])

TP, FP, FN = 0, 0, 0
iou_threshold = 0.5

for img_name in tqdm(os.listdir(image_folder)):
    if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # 1. Görseli yükle
    img_path = os.path.join(image_folder, img_name)
    image = Image.open(img_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)

    # 2. Ground truth bboxları oku
    txt_name = os.path.splitext(img_name)[0] + '.txt'
    label_path = os.path.join(label_folder, txt_name)
    if not os.path.exists(label_path):
        continue  # Etiket yoksa atla
    gt_boxes = load_labels(label_path)
    matched = np.zeros(len(gt_boxes))

    # 3. Tahmin al
    with torch.no_grad():
        outputs = model(image_tensor)
    pred_boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    pred_boxes = pred_boxes[scores > 0.5]  # eşik uygula

    # 4. IoU eşleştirme
    for pred_box in pred_boxes:
        ious = [compute_iou(pred_box, gt_box) for gt_box in gt_boxes]
        if not ious:
            FP += 1
            continue

        max_iou = max(ious)
        max_idx = np.argmax(ious)
        if max_iou >= iou_threshold and matched[max_idx] == 0:
            TP += 1
            matched[max_idx] = 1
        else:
            FP += 1

    FN += len(gt_boxes) - sum(matched)

# === Sonuçlar ===
precision = TP / (TP + FP + 1e-6)
recall = TP / (TP + FN + 1e-6)
f1 = 2 * precision * recall / (precision + recall + 1e-6)

print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

100%|██████████| 1408/1408 [03:07<00:00,  7.53it/s]

Precision: 0.7101
Recall:    0.8388
F1 Score:  0.7691
